# CONJUNTO DE EJERCICIOS 1.3
# Ejercicio 1

**Utilice aritmética de corte de tres dígitos** para calcular las siguientes sumas.  
Para cada parte, ¿qué método es más preciso y por qué?


### a)

$\sum_{i=1}^{10} \left( \frac{1}{i^2} \right)$

Primero por:

$\frac{1}{1} + \frac{1}{4} + \cdots + \frac{1}{100}$

y luego por:

$\frac{1}{100} + \frac{1}{81} + \cdots + \frac{1}{1}$


### b)


$\sum_{i=1}^{10} \left( \frac{1}{i^3} \right)$


Primero por:


$\frac{1}{1} + \frac{1}{8} + \frac{1}{27} + \cdots + \frac{1}{1000}$


y luego por:


$\frac{1}{1000} + \frac{1}{729} + \cdots + \frac{1}{1}$



**Pregunta:**  
¿Cuál de los dos métodos (de menor a mayor o de mayor a menor) es más preciso y por qué?


# Pseudocódigo: Suma con Aritmética de Corte a Tres Dígitos

**Objetivo:**  
Calcular las sumas:  
a)  Σ(1 / i²)  
b)  Σ(1 / i³)  
usando aritmética de **corte a tres dígitos significativos**,  
y comparar los resultados según el orden de suma (ascendente o descendente).

---

## Funciones auxiliares

### Función `CHOP(x, d)`
```
Entrada: x (número real), d (número de cifras significativas, por defecto 3)  
Salida: número cortado a d cifras significativas

SI x = 0 → retornar 0
ax ← |x|
k  ← piso(log10(ax))                 # exponente decimal
s  ← 10^(k - d + 1)                  # factor de escala
y  ← piso(ax / s) * s                # cortar sin redondear
SI x < 0 → y ← -y
retornar y
```

---

### Función `DIV_CHOP(a, b, d)`
```
retornar CHOP(a / b, d)
```

---

### Función `ADD_CHOP(a, b, d)`
```
retornar CHOP(a + b, d)
```

---

### Función `TERM(i, p, d)`
```
# Calcula 1 / i^p aplicando corte al final
retornar DIV_CHOP(1.0, i^p, d)
```

---

## Función principal de suma

### Función `SUMA_CHOP(p, orden, d)`
```
Entrada: p (potencia), orden ("asc" o "desc"), d (cifras significativas)
Salida: suma aproximada con corte

SI orden = "asc" → recorrer i desde 1 hasta 10
SI orden = "desc" → recorrer i desde 10 hasta 1

s ← 0
PARA cada i en el rango:
    t ← TERM(i, p, d)
    s ← ADD_CHOP(s, t, d)
retornar s
```

---

## Programa principal

```
PARA p en {2, 3} HACER:
    s_asc  ← SUMA_CHOP(p, "asc")
    s_desc ← SUMA_CHOP(p, "desc")
    s_real ← SUMA real en doble precisión (solo para comparar)

    Ea_asc  ← |s_asc - s_real|
    Ea_desc ← |s_desc - s_real|

    Mostrar:
        - Valor ascendente y su error
        - Valor descendente y su error
        - Valor real (doble precisión)
        - Qué orden es más preciso

FIN PARA
```

---

## Conclusión

- En aritmética de corte, **sumar de menor a mayor (descendente: 10 → 1)** es **más preciso**,  
  porque los términos pequeños no se pierden al sumarse a un acumulado grande.


In [1]:


from math import log10, floor

def chop_sig(x: float, d: int = 3) -> float:

    if x == 0.0:
        return 0.0
    ax = abs(x)
    k = floor(log10(ax))  # exponente decimal (p.ej., 123.4 -> 2; 0.0123 -> -2)
    s = 10.0 ** (k - d + 1)  # escala para dejar d cifras
    chopped = floor(ax / s) * s
    return chopped if x >= 0 else -chopped

def add_chop(a: float, b: float, d: int = 3) -> float:

    return chop_sig(a + b, d)

def div_chop(num: float, den: float, d: int = 3) -> float:
  
    if den == 0:
        raise ZeroDivisionError("Denominador cero")
    return chop_sig(num / den, d)

def term_inv_power(i: int, p: int, d: int = 3) -> float:
   
    return div_chop(1.0, float(i ** p), d)

def sum_chopped(p: int, ascending: bool = True, d: int = 3) -> float:

    rng = range(1, 11) if ascending else range(10, 0, -1)
    s = 0.0
    for i in rng:
        t = term_inv_power(i, p, d)
        s = add_chop(s, t, d)
    return s

def sum_true(p: int) -> float:
 
    return sum(1.0 / (i ** p) for i in range(1, 11))

def report_for(p: int, d: int = 3):
    print(f"\n=== Suma con corte a {d} cifras significativas, p={p} ===")
    s_up = sum_chopped(p, ascending=True, d=d)
    s_down = sum_chopped(p, ascending=False, d=d)
    s_true = sum_true(p)

    def errs(approx: float):
        ea = abs(approx - s_true)
        er = ea / abs(s_true)
        return ea, er

    ea_up, er_up = errs(s_up)
    ea_down, er_down = errs(s_down)

    print(f"Orden ascendente (1→10): {s_up:.10f} | Ea={ea_up:.10e} | Er={er_up:.10%}")
    print(f"Orden descendente (10→1): {s_down:.10f} | Ea={ea_down:.10e} | Er={er_down:.10%}")
    print(f"Valor 'real' (doble precisión): {s_true:.10f}")

    better = "ascendente" if ea_up < ea_down else ("descendente" if ea_down < ea_up else "empate")
    print(f"Más preciso: {better}")

if __name__ == "__main__":
    # Parte (a): p=2
    report_for(p=2, d=3)
    # Parte (b): p=3
    report_for(p=3, d=3)



=== Suma con corte a 3 cifras significativas, p=2 ===
Orden ascendente (1→10): 1.5300000000 | Ea=1.9767731167e-02 | Er=1.2755286337%
Orden descendente (10→1): 1.5400000000 | Ea=9.7677311665e-03 | Er=0.6302706509%
Valor 'real' (doble precisión): 1.5497677312
Más preciso: descendente

=== Suma con corte a 3 cifras significativas, p=3 ===
Orden ascendente (1→10): 1.1600000000 | Ea=3.7531985674e-02 | Er=3.1341113326%
Orden descendente (10→1): 1.1900000000 | Ea=7.5319856742e-03 | Er=0.6289590394%
Valor 'real' (doble precisión): 1.1975319857
Más preciso: descendente


# Ejercicio 2

La serie de Maclaurin para la función arctangente converge para $-1 < x \le 1$ y está dada por:

$ \displaystyle \arctan x = \lim_{n \to \infty} P_n(x) = \lim_{n \to \infty} \sum_{i=1}^{n} (-1)^{i+1} \frac{x^{2i-1}}{2i-1} $

---

### a)

Utilice el hecho de que $ \tan\!\left(\frac{\pi}{4}\right) = 1 $ para determinar el número $n$ de términos de la serie que se necesita sumar para garantizar que

$ \displaystyle \left|\,4P_n(1) - \pi\,\right| < 10^{-3} $

---

### b)

El lenguaje de programación **C++** requiere que el valor de $\pi$ se encuentre dentro de $10^{-10}$.  
¿Cuántos términos de la serie se necesitarían sumar para obtener este grado de precisión?

---

**Nota:** La serie de Maclaurin para $\arctan(1)$ es la **serie de Leibniz para $\pi$**:

$ \displaystyle \frac{\pi}{4} = 1 - \frac{1}{3} + \frac{1}{5} - \frac{1}{7} + \frac{1}{9} - \cdots $




## Pseudocódigo

**Inicio**  
1. Definir función $n\_min(\varepsilon)$:  
   $n \leftarrow \lceil (\dfrac{1}{\varepsilon} - 1) / 2 \rceil$
   devolver $n$.
2. Definir función $sumar\_Pn(n)$:  
   $acum \leftarrow 0$  
   Para $i$ desde $1$ hasta $n$ hacer  
   &nbsp;&nbsp;&nbsp; $termino \leftarrow (-1)^{i+1} * \dfrac{1}{2i-1}$  
   &nbsp;&nbsp;&nbsp; $acum \leftarrow acum + termino$  
   Fin Para  
   devolver $4 * acum$
3. Para $\varepsilon = 10^{-3}$:  
   $n \leftarrow n\_min(10^{-3})$  
   $pi\_aprox \leftarrow sumar\_Pn(n)$  
   Mostrar $n$ y $pi\_aprox$.
4. Para $\varepsilon = 10^{-10}$:  
   $n \leftarrow n\_min(10^{-10})$  
   (Opcional) $pi\_aprox \leftarrow sumar\_Pn(n)$  
   Mostrar $n$ (y $pi\_aprox$ si se calcula).  
**Fin**


In [3]:
import math

def n_min(eps: float) -> int:
 
    return math.ceil(((4.0/eps) - 1.0) / 2.0)

def aproximar_pi_por_arctan1(n: int) -> float:

    s = 0.0
    signo = 1.0
    for i in range(1, n+1):
        s += signo * (1.0 / (2*i - 1))
        signo *= -1.0
    return 4.0 * s

if __name__ == "__main__":
    # Literal (a)
    eps_a = 1e-3
    n_a = n_min(eps_a)
    pi_a = aproximar_pi_por_arctan1(n_a)
    print("(a) eps = 1e-3") 
    print("n mínimo:", n_a)
    print("π aproximada:", pi_a)
    print("Cota teórica del error < eps?:", 1.0/(2*n_a+1) * 4 < eps_a)

    # Literal (b)
    eps_b = 1e-10
    n_b = n_min(eps_b)
    print("\n(b) eps = 1e-10")
    print("n mínimo:", n_b)



(a) eps = 1e-3
n mínimo: 2000
π aproximada: 3.1410926536210413
Cota teórica del error < eps?: True

(b) eps = 1e-10
n mínimo: 20000000000


# Ejercicio 3

Otra fórmula para calcular $\pi$ se puede deducir a partir de la identidad

$ \displaystyle \frac{\pi}{4} = 4 \arctan\!\left(\frac{1}{5}\right) - \arctan\!\left(\frac{1}{239}\right). $

Determine el número de términos que se deben sumar para garantizar una aproximación de $\pi$ dentro de $10^{-3}$.


## Pseudocódigo

**Inicio**  
1. Definir función $cota(n)$:  
   devolver $\dfrac{16\,(1/5)^{2n+1} + 4\,(1/239)^{2n+1}}{2n+1}$.
2. Definir $n \leftarrow 1$.  
3. Mientras $cota(n) \ge 10^{-3}$ hacer:  
   $n \leftarrow n + 1$.
4. (Opcional) Calcular la aproximación:  
   $\pi\_{\text{aprox}} \leftarrow 4\Bigg(4\sum\limits_{i=1}^{n} (-1)^{i+1}\dfrac{(1/5)^{2i-1}}{2i-1} \; - \; \sum\limits_{i=1}^{n} (-1)^{i+1}\dfrac{(1/239)^{2i-1}}{2i-1}\Bigg)$
5. Mostrar $n$, la $cota(n)$ y (opcional) $\pi\_{\text{aprox}}$.  
**Fin**

In [ ]:
import math

def cota_error(n: int) -> float:
 
    return (16.0 * (1/5)**(2*n + 1) + 4.0 * (1/239)**(2*n + 1)) / (2*n + 1)

def n_min_para_eps(eps: float = 1e-3) -> int:
    n = 1
    while cota_error(n) >= eps:
        n += 1
    return n

def aproximar_pi(n: int) -> float:
    def S(n: int, x: float) -> float:
        s = 0.0
        signo = 1.0
        for i in range(1, n+1):
            s += signo * (x ** (2*i - 1)) / (2*i - 1)
            signo *= -1.0
        return s
    return 4.0 * (4.0 * S(n, 1/5) - S(n, 1/239))

if __name__ == "__main__":
    eps = 1e-3
    n = n_min_para_eps(eps)
    pi_aprox = aproximar_pi(n)
    print("Tolerancia eps:", eps)
    print("n mínimo:", n)
    print("Cota teórica del error:", cota_error(n))

    print("Error real |pi_aprox - pi|:", abs(pi_aprox - math.pi))


Tolerancia eps: 0.001
n mínimo: 3
Cota teórica del error: 2.9257142857155695e-05
Error real |pi_aprox - pi|: 2.837573524150372e-05


# Ejercicio 4

Compare los siguientes tres algoritmos.  
¿Cuándo es correcto el algoritmo de la parte 1a?

---

### a)

**ENTRADA:** $n, x_1, x_2, \dots, x_n$  
**SALIDA:** `PRODUCT`

**Paso 1:** Determine `PRODUCT = 0`.  
**Paso 2:** Para $i = 1, 2, \dots, n$ haga  
&emsp;Determine `PRODUCT = PRODUCT * x_i`.  
**Paso 3:** SALIDA `PRODUCT`;  
**PARE.**

---

### b)

**ENTRADA:** $n, x_1, x_2, \dots, x_n$  
**SALIDA:** `PRODUCT`

**Paso 1:** Determine `PRODUCT = 1`.  
**Paso 2:** Para $i = 1, 2, \dots, n$ haga  
&emsp;Set `PRODUCT = PRODUCT * x_i`.  
**Paso 3:** SALIDA `PRODUCT`;  
**PARE.**

---

### c)

**ENTRADA:** $n, x_1, x_2, \dots, x_n$  
**SALIDA:** `PRODUCT`

**Paso 1:** Determine `PRODUCT = 1`.  
**Paso 2:** Para $i = 1, 2, \dots, n$ haga  
&emsp;Si $x_i = 0$ entonces determine `PRODUCT = 0`;  
&emsp;SALIDA `PRODUCT`;  
&emsp;**PARE.**  
&emsp;Determine `PRODUCT = PRODUCT * x_i`.  
**Paso 3:** SALIDA `PRODUCT`;  
**PARE.**

---

**Pregunta:**  
¿Cuándo es correcto el algoritmo de la parte (a)?


**Entrada:** $n$, arreglo $x[1..n]$  
**Salida:** `PRODUCT` de acuerdo con cada algoritmo

### (a) — Siempre $0$ salvo que haya ceros (único caso en que coincide con el producto real)
1. $PRODUCT \leftarrow 0$
2. Para $i \leftarrow 1$ hasta $n$ hacer  
   $PRODUCT \leftarrow PRODUCT * x[i]$
3. Salida: $PRODUCT$

### (b) — Producto correcto estándar
1. $PRODUCT \leftarrow 1$
2. Para $i \leftarrow 1$ hasta $n$ hacer  
   $PRODUCT \leftarrow PRODUCT * x[i]$
3. Salida: $PRODUCT$

### (c) — Producto con corte temprano (óptimo cuando hay ceros)
1. $PRODUCT \leftarrow 1$
2. Para $i \leftarrow 1$ hasta $n$ hacer  
   Si $x[i] = 0$ entonces:  
   &nbsp;&nbsp;a. $PRODUCT \leftarrow 0$  
   &nbsp;&nbsp;b. Salida: $PRODUCT$ y **pare**  
   Si no: $PRODUCT \leftarrow PRODUCT * x[i]$
3. Salida: $PRODUCT$



In [5]:
from typing import List

def algoritmo_a(xs: List[float]) -> float:
    product = 0.0
    for x in xs:
        product = product * x
    return product

def algoritmo_b(xs: List[float]) -> float:
    product = 1.0
    for x in xs:
        product *= x
    return product

def algoritmo_c(xs: List[float]) -> float:
    product = 1.0
    for x in xs:
        if x == 0:
            return 0.0  # corte temprano
        product *= x
    return product

def es_correcto_a(xs: List[float]) -> bool:
   
    return any(x == 0 for x in xs)

if __name__ == "__main__":
    # Entrada interactiva simple
    try:
        n = int(input("Ingrese n: ").strip())
        xs = []
        for i in range(1, n+1):
            xs.append(float(input(f"x[{i}] = ").strip()))
    except Exception:
        print("Entrada inválida.")
        raise SystemExit(1)

    real_product = algoritmo_b(xs)  # producto correcto
    pa = algoritmo_a(xs)
    pb = real_product
    pc = algoritmo_c(xs)

    print("Producto real (b):", pb)
    print("Resultado algoritmo (a):", pa)
    print("Resultado algoritmo (c):", pc)
    print("¿Cuándo es correcto el (a)? ->", "Con algún x_i = 0" )
    print("¿(a) es correcto para este conjunto? ", es_correcto_a(xs))


Producto real (b): 144.0
Resultado algoritmo (a): 0.0
Resultado algoritmo (c): 144.0
¿Cuándo es correcto el (a)? -> Con algún x_i = 0
¿(a) es correcto para este conjunto?  False


# Ejercicio 5

### a)

¿Cuántas multiplicaciones y sumas se requieren para determinar una suma de la forma:

$$ \displaystyle \sum_{i=1}^{n} \sum_{j=1}^{i} a_i b_j ? $$

---

### b)

Modifique la suma en la parte (a) a un formato equivalente que reduzca el número de cálculos.


**Entrada:** $n$, arreglos $a[1..n]$, $b[1..n]$  
**Salida:** `S` = $\sum_{i=1}^{n}\sum_{j=1}^{i} a_i b_j$

1. $B_1 \leftarrow b_1$
2. Para $i \leftarrow 2$ hasta $n$ hacer  
   $B_i \leftarrow B_{i-1} + b_i$
3. $S \leftarrow 0$
4. Para $i \leftarrow 1$ hasta $n$ hacer  
   $S \leftarrow S + a_i * B_i$
5. Salida `S`

**Conteos teóricos:**  
- Multiplicaciones: $n$  
- Sumas: $2n-2$

In [6]:
from typing import List, Tuple

def conteo_directo(n: int) -> Tuple[int, int]:
    """(a) Conteo teórico de operaciones para la evaluación directa."""
    mults = n * (n + 1) // 2
    sums = mults - 1 if mults > 0 else 0
    return mults, sums

def suma_directa(a: List[float], b: List[float]) -> float:
    """Evaluación directa (O(n^2))."""
    s = 0.0
    n = len(a)
    for i in range(n):
        for j in range(i + 1):
            s += a[i] * b[j]
    return s

def suma_optimizada(a: List[float], b: List[float]) -> float:
    """(b) Usa prefijos de b: B_i = sum_{j=1..i} b_j, luego suma a_i * B_i (O(n))."""
    n = len(a)
    if n == 0:
        return 0.0
    # Prefijos de b
    B_prev = b[0]
    S = a[0] * B_prev
    for i in range(1, n):
        B_prev += b[i]
        S += a[i] * B_prev
    return S

def conteo_optimizado(n: int) -> Tuple[int, int]:
    """(b) Conteo teórico usando prefijos."""
    mults = n
    sums = (n - 1) + (n - 1) if n >= 1 else 0  # prefijos + acumulación
    return mults, sums

if __name__ == "__main__":
    try:
        n = int(input("Ingrese n: ").strip())
        a = list(map(float, input("Ingrese a[1..n] separados por espacio: ").strip().split()))
        b = list(map(float, input("Ingrese b[1..n] separados por espacio: ").strip().split()))
        assert len(a) == n and len(b) == n
    except Exception:
        print("Entrada inválida: asegúrese de dar n y dos listas de longitud n.")
        raise SystemExit(1)

    S_dir = suma_directa(a, b)
    S_opt = suma_optimizada(a, b)
    m_dir, s_dir = conteo_directo(n)
    m_opt, s_opt = conteo_optimizado(n)

    print("Resultado directo:   ", S_dir)
    print("Resultado optimizado: ", S_opt)
    print("¿Coinciden? ", abs(S_dir - S_opt) < 1e-12)
    print("(a) Conteos teóricos -> mult:", m_dir, ", sumas:", s_dir)
    print("(b) Conteos teóricos -> mult:", m_opt, ", sumas:", s_opt)


Resultado directo:    290.0
Resultado optimizado:  290.0
¿Coinciden?  True
(a) Conteos teóricos -> mult: 15 , sumas: 14
(b) Conteos teóricos -> mult: 5 , sumas: 8


# Discusiones 





### 1.

Escriba un algoritmo para sumar la serie finita  
$ \displaystyle \sum_{i=1}^{n} x_i $  
en orden inverso.




**Entrada:** $n$, arreglo $x[1..n]$  
**Salida:** `S` = $\sum_{i=1}^{n} x_i$

1. $S \leftarrow 0$
2. Para $i \leftarrow n$ hasta $1$ con paso $-1$ hacer  
   $S \leftarrow S + x[i]$
3. Salida: $S$


In [7]:
from typing import List

def suma_inversa(xs: List[float]) -> float:

    S = 0.0

    for i in range(len(xs) - 1, -1, -1):
        S += xs[i]
    return S

if __name__ == "__main__":
    try:
        n = int(input("Ingrese n: ").strip())
        xs = list(map(float, input("Ingrese los valores x1..xn separados por espacio: ").strip().split()))
        assert len(xs) == n
    except Exception:
        print("Entrada inválida. Asegúrese de ingresar n y la lista completa.")
        raise SystemExit(1)

    total = suma_inversa(xs)
    print("La suma en orden inverso es:", total)


La suma en orden inverso es: 15.0


### 2.

Las ecuaciones (1.2) y (1.3) en la sección 1.2 proporcionan formas alternativas para las raíces $x_1$ y $x_2$ de  
$ ax^2 + bx + c = 0 $.

Construya un algoritmo con entrada $a$, $b$, $c$ y salida $x_1$, $x_2$ que calcule las raíces $x_1$, $x_2$ (que pueden ser iguales o conjugadas complejas) mediante la mejor fórmula para cada raíz.





**Inicio**  
1. Si $a = 0$ entonces:  
   1.1 Si $b \ne 0$: devolver $x_1 = x_2 = -c/b$.  
   1.2 Si $b = 0$ entonces:  
   &nbsp;&nbsp;• Si $c = 0$: devolver “infinitas soluciones”.  
   &nbsp;&nbsp;• En caso contrario: devolver “sin solución”.  
2. Calcular $\Delta \leftarrow b*b - 4*a*c$.  
3. Calcular $r \leftarrow \sqrt{\Delta}$ (permitiendo valor complejo).  
4. Si $b \ge 0$ entonces $q \leftarrow -0.5\,(b + r)$; en caso contrario $q \leftarrow -0.5\,(b - r)$.  
5. Calcular $x_1 \leftarrow q/a$.  
6. Si $q \ne 0$ entonces $x_2 \leftarrow c/q$; si $q = 0$ usar $x_2 \leftarrow (-b - r)/(2a)$ (respaldo).  
7. Devolver $x_1, x_2$.  
**Fin**

In [8]:
import math
import cmath
from typing import Tuple, Union

Number = Union[float, complex]

def roots_best(a: Number, b: Number, c: Number) -> Tuple[Number, Number, str]:
   
    if a == 0:
        if b != 0:
            x = -c / b
            return x, x, "lineal"
        else:
            return complex("nan"), complex("nan"), "indeterminado" if c == 0 else "sin_solucion"

  
    Delta = b*b - 4*a*c

 
    r = math.sqrt(Delta) if Delta >= 0 else cmath.sqrt(Delta)

   
    if b >= 0:
        q = -0.5 * (b + r)
    else:
        q = -0.5 * (b - r)

    x1 = q / a
    if q != 0:
        x2 = c / q
    else:
        
        denom = 2*a
        x2 = (-b - r) / denom
    return x1, x2, ("real" if Delta >= 0 else "compleja")

if __name__ == "__main__":
    try:
        a = float(input("a = ").strip())
        b = float(input("b = ").strip())
        c = float(input("c = ").strip())
    except Exception:
        print("Entrada inválida.")
        raise SystemExit(1)

    x1, x2, estado = roots_best(a, b, c)
    print("Estado:", estado)
    print("x1 =", x1)
    print("x2 =", x2)


Estado: compleja
x1 = (-0.6-0.6633249580710799j)
x2 = (-0.6+0.6633249580710799j)


### 3.

Suponga que

$ \displaystyle \frac{1 - 2x}{1 - x + x^2} + \frac{2x - 4x^3}{1 - x + x^4} + \frac{4x^3 - 8x^7}{1 - x + x^8} + \cdots = \frac{1 + 2x}{1 + x + x^2} $

para $x < 1$ y si $x = 0.25$.

Escriba y ejecute un algoritmo que determine el número de términos necesarios en el lado izquierdo de la ecuación de tal forma que el lado izquierdo difiera del lado derecho en menos de $10^{-6}$.

**Entrada:** $x=0.25$, $\varepsilon=10^{-6}$  
**Salida:** $N$ mínimo y $S_N(x)$

1. $R \leftarrow \dfrac{1+2x}{1+x+x^2}$
2. $S \leftarrow 0$; $k \leftarrow 0$
3. Mientras $|S - R| \ge \varepsilon$ hacer
   - $num \leftarrow 2^{k} x^{2^{k}-1} - 2^{k+1} x^{2^{k+1}-1}$
   - $den \leftarrow 1 - x^{2^{k}} + x^{2^{k+1}}$
   - $S \leftarrow S + num/den$
   - $k \leftarrow k + 1$
4. Devolver $N \leftarrow k$, $S$ y el error $|S-R|$  
**Fin**

In [9]:
import math

def termino_k(x: float, k: int) -> float:
    num = (2**k) * (x ** (2**k - 1)) - (2**(k+1)) * (x ** (2**(k+1) - 1))
    den = 1.0 - (x ** (2**k)) + (x ** (2**(k+1)))
    return num / den

def sumar_hasta_eps(x: float, eps: float = 1e-6):
    R = (1 + 2*x) / (1 + x + x*x)
    S = 0.0
    k = 0
    while abs(S - R) >= eps:
        S += termino_k(x, k)
        k += 1
        # por seguridad, límite de iteraciones
        if k > 100:
            break
    return k, S, R, abs(S - R)

if __name__ == "__main__":
    x = 0.25
    N, S, R, err = sumar_hasta_eps(x, eps=1e-6)
    print("x =", x)
    print("N mínimo =", N)
    print("S_N =", S)
    print("RHS =", R)
    print("Error |S_N - RHS| =", err)


x = 0.25
N mínimo = 4
S_N = 1.1428571279559818
RHS = 1.1428571428571428
Error |S_N - RHS| = 1.4901160971803051e-08
